In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)

net(X)

tensor([[ 0.0074, -0.2072, -0.1328,  0.0846,  0.0666,  0.1814, -0.0815, -0.2136,
          0.1400, -0.0008],
        [ 0.1475, -0.2156, -0.0562,  0.0725,  0.0282,  0.2446,  0.0476, -0.2073,
          0.2016,  0.0485]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(20, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, X):
        return self.mlp(X)

In [5]:
net = MLP()
net(X)

tensor([[ 0.0294, -0.0684,  0.0235, -0.0921,  0.1370, -0.1397,  0.1768, -0.0199,
          0.0356, -0.0083],
        [-0.0436, -0.1782, -0.1315,  0.0191,  0.0213, -0.1048,  0.0509, -0.0555,
         -0.0838, -0.0431]], grad_fn=<AddmmBackward0>)

In [8]:
class MySequential(nn.Module):

    def __init__(self, *args):
        super(MySequential, self).__init__()
        
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1357,  0.0054, -0.0020,  0.2741, -0.1998, -0.1720, -0.1318,  0.2814,
         -0.0240, -0.1381],
        [-0.1255, -0.1085, -0.1350,  0.3370, -0.1716, -0.0797, -0.2120,  0.2272,
         -0.0177, -0.1306]], grad_fn=<AddmmBackward0>)

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP,self).__init__()

        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [13]:
net = FixedHiddenMLP()
net(X)

tensor(0.5022, grad_fn=<SumBackward0>)